# Tutorial 4 : Attentive Neural Processes Variants - 1D GP Data

Last Update : 12 June 2019

**Aim**: 
- Investigating variants of [Attentive Neural Process] (ANP)
- Showing how to use the library to build more complicated models

**Nota Bene:**
- The majority of the work below does not follow any paper
- Much more details about the framework and dataset can be found in [Tutorial 1 - Conditional Neural Process].

[Attentive Neural Process]: https://arxiv.org/abs/1901.05761
[Tutorial 1 - Conditional Neural Process]: Tutorial%201%20-%20Conditional%20Neural%20Process.ipynb

**Environment Hypermarameters:**

In [1]:
N_THREADS = 8
# Nota Bene : notebooks don't deallocate GPU memory
IS_FORCE_CPU = False # can also be set in the trainer

## Environment

In [2]:
cd ..

/master


In [3]:
%autosave 600
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# CENTER PLOTS
from IPython.core.display import HTML
display(HTML(""" <style> .output_png {display: table-cell; text-align: center; margin:auto; }
.prompt display:none;}  </style>"""))

import os
if IS_FORCE_CPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = ""

import sys
sys.path.append("notebooks")

import numpy as np
import matplotlib.pyplot as plt
import torch
torch.set_num_threads(N_THREADS)

Autosaving every 600 seconds


## Dataset

In this notebook I only look at the periodic and mattern kernel as these seem the harder to learn. See [Tutorial 1 - Conditional Neural Process] for more details.

[Tutorial 1 - Conditional Neural Process]: Tutorial%201%20-%20Conditional%20Neural%20Process.ipynb

In [4]:
from sklearn.gaussian_process.kernels import RBF, Matern, ExpSineSquared, DotProduct, ConstantKernel
from ntbks_viz import plot_posterior_samples, plot_prior_samples, plot_dataset_samples

from ntbks_datasets import GPDataset

In [5]:
X_DIM = 1  # 1D spatial input
Y_DIM = 1  # 1D regression
N_POINTS = 128
N_SAMPLES = 100000 # this is a lot and can work with less

In [6]:
datasets = dict()
kwargs = dict(n_samples=N_SAMPLES, n_points=N_POINTS)
datasets["matern"] = GPDataset(kernel=1.0 * Matern(length_scale=1.0,
                                                   length_scale_bounds=(1e-1, 10.0),
                                                   nu=1.5),
                               **kwargs)
datasets["periodic"] = GPDataset(kernel=1.0 * ExpSineSquared(length_scale=1.0,
                                                             periodicity=3.0,
                                                             length_scale_bounds=(0.1, 10.0),
                                                             periodicity_bounds=(1.0, 10.0)),
                                 **kwargs)

## Model

The general model architecture is slightly different from the paper to make it modular and easy to extend, but it is easy to make them equivalent with the right parameters. Refer to [Tutorial 1 - Conditional Neural Process] for an overview main parameters or the docstrings of `NeuralProcess` for all parameters. 

[Tutorial 1 - Conditional Neural Process]: Tutorial%201%20-%20Conditional%20Neural%20Process.ipynb

In [7]:
from torch.distributions import Normal

from skssl.transformers import (NeuralProcessLoss, NeuralProcess, AttentiveNeuralProcess, 
                                SelfAttentionBlock, SinusoidalEncodings)
from skssl.predefined import MLP, merge_flat_input, get_uninitialized_mlp
from skssl.transformers.neuralproc.datasplit import context_target_split

In [8]:
R_DIM = 128
RANGE_CNTXT = (4, 50)  # context points will be sampled uniformly in this range
RANGE_EXTRA_TRGTS = (3, N_POINTS-RANGE_CNTXT[1])  # extra number of targtes points 

In [9]:
def get_cntxt_trgt(*args):
    return context_target_split(*args, range_cntxts=RANGE_CNTXT, range_extra_trgts=RANGE_EXTRA_TRGTS)

def get_attn_bloc(attention):
    return lambda *args: SelfAttentionBlock(*args, attention=attention)

def init_models():
    return {"Conditional_ANP":AttentiveNeuralProcess(X_DIM, Y_DIM,
                                  r_dim=R_DIM,
                                  get_cntxt_trgt=get_cntxt_trgt,
                                  encoded_path="deterministic"),
     
     "Multihead_ANP":AttentiveNeuralProcess(X_DIM, Y_DIM,
                                  r_dim=R_DIM,
                                  get_cntxt_trgt=get_cntxt_trgt,
                                  attention="multihead", 
                                  encoded_path="both") ,
     
     "Sinusoidal_ANP":AttentiveNeuralProcess(X_DIM, Y_DIM,
                                  XEncoder=SinusoidalEncodings, # sinusoidal encodings
                                  r_dim=R_DIM,
                                  get_cntxt_trgt=get_cntxt_trgt,
                                  encoded_path="both") ,
     
     "SelfAttn_Enc_ANP":AttentiveNeuralProcess(X_DIM, Y_DIM,
                                  XYEncoder=SelfAttentionBlock,  # self attention encoder 
                                  r_dim=R_DIM,
                                  get_cntxt_trgt=get_cntxt_trgt,
                                  encoded_path="both") ,
     
     "SelfAttn_Dec_ANP":AttentiveNeuralProcess(X_DIM, Y_DIM,
                                  Decoder=SelfAttentionBlock,  # self attention decoder 
                                  r_dim=R_DIM,
                                  get_cntxt_trgt=get_cntxt_trgt,
                                  encoded_path="both"),
     
     "r128_Conditional_Transformer_Process":AttentiveNeuralProcess(X_DIM, Y_DIM,
                                  XEncoder=SinusoidalEncodings, # sinusoidal encodings
                                  XYEncoder=get_attn_bloc("transformer"),  # self attention encoder 
                                  Decoder=get_attn_bloc("transformer"),  # self attention decoder 
                                  r_dim=128,
                                  get_cntxt_trgt=get_cntxt_trgt,
                                  attention="transformer", 
                                  encoded_path="deterministic"),
     
     "r64_Conditional_Transformer_Process":AttentiveNeuralProcess(X_DIM, Y_DIM,
                                  XEncoder=SinusoidalEncodings, # sinusoidal encodings
                                  XYEncoder=get_attn_bloc("transformer"),  # self attention encoder 
                                  Decoder=get_attn_bloc("transformer"),  # self attention decoder 
                                  r_dim=64,
                                  get_cntxt_trgt=get_cntxt_trgt,
                                  attention="transformer", 
                                  encoded_path="deterministic")}

# initialize all model for each dataset
data_models = {name: (init_models(), data) 
               for name, data in datasets.items()}

/master/skssl/predefined/mlp.py:56: UserWarning: hidden_size=32 smaller than output=128 and input=128. Setting it to 128.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))
/master/skssl/predefined/mlp.py:56: UserWarning: hidden_size=32 smaller than output=256 and input=128. Setting it to 128.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))


### N Param

Number of parameters (note that I did not play around with this much, this depends a lot on the representation size):

In [10]:
from utils.helpers import count_parameters

In [11]:
for k, (models, dataset) in data_models.items():
    for name, model in models.items():
        print("N Param for {}:".format(name), count_parameters(model))
    break

N Param for Conditional_ANP: 98498
N Param for Multihead_ANP: 246594
N Param for Sinusoidal_ANP: 176642
N Param for SelfAttn_Enc_ANP: 296770
N Param for SelfAttn_Dec_ANP: 306178
N Param for r128_Conditional_Transformer_Process: 417986
N Param for r64_Conditional_Transformer_Process: 106626


## Training

In [12]:
from skorch.callbacks import ProgressBar, Checkpoint

from skssl.training import NeuralNetTransformer
from skssl.training.helpers import make_Xy_input

In [13]:
N_EPOCHS = 50
BATCH_SIZE = 64 
is_RETRAIN = False # if false load precomputed

In [ ]:
for k,(models, dataset) in data_models.items():
    for name, neural_proc in models.items():
        
        print()
        print("--- {} {} ---".format("Training" if is_RETRAIN else "Loading", k + " " + name))
        print()

        chckpt = Checkpoint(dirname="results/notebooks/neural_process/variants/{}_{}".format(name, k), 
                            monitor='train_loss_best') # train would be same as validation as always resamples

        model = NeuralNetTransformer(neural_proc, NeuralProcessLoss,
                                     max_epochs=N_EPOCHS,
                                     batch_size=BATCH_SIZE,
                                     train_split=None,  # don't use cross validation dev set
                                     lr=1e-3, 
                                     callbacks=[ProgressBar(), 
                                                chckpt], # checkpoint best model
                                    ) 

        if is_RETRAIN:
            _=model.fit(*make_Xy_input(dataset))

        model.initialize()
        model.load_params(checkpoint=chckpt)


--- Training matern Conditional_ANP ---



  epoch    train_loss    cp      dur
-------  ------------  ----  -------
      1      110.6675     +  24.0356


      2       67.8055     +  22.5172


      3       45.0732     +  24.3130


      4       28.6640     +  24.1081


...

     47      -26.5755        34.6914


     48      -20.2154        36.1948


     49      -24.6033        34.8502


     50      -23.1066        39.7006
Re-initializing optimizer because the following parameters were re-set: .

--- Training matern r64_Conditional_Transformer_Process ---



  epoch    train_loss    cp      dur
-------  ------------  ----  -------
      1      120.2381     +  38.2808


# Trained Prior

In [ ]:
EXTRAP_DISTANCE = 2  # add 2 to the right for extrapolation
INTERPOLATION_RANGE = dataset.min_max
EXTRAPOLATION_RANGE = (dataset.min_max[0], dataset.min_max[1]+EXTRAP_DISTANCE )

In [ ]:
for k,(neural_proc, dataset) in data_models.items():
    for name, neural_proc in models.items():
        plot_prior_samples(neural_proc, 
                           title="Trained Prior Samples : {} {}".format(name, k), 
                           test_min_max=EXTRAPOLATION_RANGE, 
                           train_min_max=INTERPOLATION_RANGE)

# Posterior

In [ ]:
N_CNTXT = 10
for k,(neural_proc, dataset) in data_models.items():
    for name, neural_proc in models.items():
        plot_posterior_samples(dataset, neural_proc, 
                               n_cntxt=N_CNTXT, 
                               test_min_max=EXTRAPOLATION_RANGE, 
                               n_points=2*N_POINTS,
                               title="Posterior Samples Conditioned on {} Context Points : {} {}".format(N_CNTXT, name, k))

In [ ]:
N_CNTXT = 2
for k,(neural_proc, dataset) in data_models.items():
    for name, neural_proc in models.items():
        plot_posterior_samples(dataset, neural_proc, 
                               n_cntxt=N_CNTXT, 
                               test_min_max=EXTRAPOLATION_RANGE, 
                               n_points=2*N_POINTS,
                               title="Posterior Samples Conditioned on {} Context Points : {} {}".format(N_CNTXT, name, k))

In [ ]:
N_CNTXT = 20
for k,(neural_proc, dataset) in data_models.items():
    for name, neural_proc in models.items():
        plot_posterior_samples(dataset, neural_proc, 
                               n_cntxt=N_CNTXT, 
                               test_min_max=EXTRAPOLATION_RANGE, 
                               n_points=2*N_POINTS,
                               title="Posterior Samples Conditioned on {} Context Points : {} {}".format(N_CNTXT, name, k))